**Welcome to AhmadRAG**

In [ ]:
import os
import re
from pypdf import PdfReader
from langchain import hub
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_ollama import OllamaLLM, OllamaEmbeddings



<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>بخش ۰: تنظیم و بارگذاری مدل‌های Ollama</title>
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB; /* رنگ پس‌زمینه بلوک‌های کد */
            padding: 10px;
            border-radius: 5px;
            direction: ltr; /* جهت متنی کدها از چپ به راست */
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 14px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul, ol {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        ul ul, ul ol, ol ul, ol ol {
            padding-right: 20px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b>بخش ۰: تنظیم و بارگذاری مدل‌های Ollama (LLM و Embeddings)</b>
        <pre>
llm_model = "llama3.1"
llm_url = "http://127.0.0.1:11434"
#Loading The LLM (Language Model)
llm = OllamaLLM(model=llm_model, base_url=llm_url)
#Setting Ollama Embeddings
 http://localhost:11434/api/tags
embed_model = OllamaEmbeddings(model=llm_model, base_url=llm_url)
        </pre>
        <p><b>توضیح:</b></p>
        <ul>
            <li><b>هدف:</b> تنظیم و بارگذاری مدل زبانی بزرگ (LLM) و امبدینگ‌های Ollama برای استفاده در پردازش‌های بعدی.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>llm_model</code>: نام مدل زبانی مورد استفاده، در اینجا "llama3.1".</li>
                    <li><code>llm_url</code>: آدرس URL برای دسترسی به مدل زبانی، معمولاً به صورت محلی.</li>
                    <li><code>OllamaLLM</code>: کلاس یا تابعی که برای بارگذاری مدل زبانی Ollama استفاده می‌شود.</li>
                    <li><code>OllamaEmbeddings</code>: کلاس یا تابعی که برای تنظیم امبدینگ‌های Ollama استفاده می‌شود.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>تعیین نام مدل و URL مربوط به مدل زبانی.</li>
                    <li>ایجاد نمونه‌ای از مدل زبانی با استفاده از <code>OllamaLLM</code> و تنظیم URL پایه.</li>
                    <li>تنظیم امبدینگ‌های Ollama با استفاده از <code>OllamaEmbeddings</code> و همان نام مدل و URL پایه.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [ ]:
llm_model= "llama3.1"
llm_url = "http://127.0.0.1:11434"

#Loading The LLM (Language Model)
llm = OllamaLLM(model=llm_model, base_url=llm_url)

#Setting Ollama Embeddings
# http://localhost:11434/api/tags
embed_model = OllamaEmbeddings(model=llm_model,base_url=llm_url)



In [4]:
# # Loading PDF Document and split

# def load_pdf(file_path):
#     reader = PdfReader(file_path)
#     text = ""
#     for page in reader.pages:
#         text += page.extract_text()
#     return text

# def split_text(text: str):
#     split_text = re.split('\n \n', text)
#     return [i for i in split_text if i != ""]


In [5]:
#Loading Text

with open('txt.md', 'r', encoding='utf-8') as file:
    file_content = file.read()

In [16]:
file_content

'\nبه گزارش گروه اجتماعی خبرگزاری تسنیم ، قدرت خاوری؛ رئیس انجمن\n کاردرمانی ایران در آیین افتتاحیه بیست\u200cوپنجمین کنگره ملی کاردرمانی\n که در سالن قلم مرکز همایش\u200cهای کتابخانه ملی ایران برگزار شد، اظهار کرد:\n کاردرمانی بیش از 50 سال قدمت دارد و در حال حاضر حدود 6 هزار نفر فارغ تحصیل کادردرمانی داریم.\n\nوی افزود: کاردرمانی از جمله رشته\u200cهای توانبخشی است که امروزه در ایران نقش بسزایی در بازتوانی حرفه\u200cای\n و فیزیکی افراد ناتوان ایفا می\u200cکند، کاردرمانگران با استفاده از روش\u200cهای خلاقانه\n، حرفه\u200cای و آکادمیک، بدنه اصلی تیم توانبخشی بسیاری از\n ضایعات نظیر فلج مغزی سکته مغزی و ... را تشکیل می\u200cدهند.\n\nخاوری با بیان اینکه کاردرمانگر علاوه بر درمان جسم، در حوزه اختلالات روانی و \nاعتیاد نیز خدمات موثری را ارائه می\u200cدهد، ادامه داد: کاردرمانی چند حوزه\n فعالیت یک انسان را دربر می\u200cگیرد که شامل بازی، خواب، شغل افراد،\n آموزش، فعالیت\u200cهای روزمره، مشارکت\u200cهای اجتماعی و مراقبت از خود است.\n\nبه گفته وی، در هریک از حیطه\u200cهای یاد شده اگر اختلال ا

In [6]:
# file_content = load_pdf(file_path="ebk.pdf")
# len(file_content)

In [1]:
# import re

# def clean_text(text):
#     # حذف کاراکترهای نامعتبر و تبدیل به UTF-8
#     return text.encode('utf-8', 'ignore').decode('utf-8')

# def remove_special_characters(text):
#     # حذف کاراکترهای غیر ASCII و کاراکتر NULL
#     text = re.sub(r'[^\x20-\x7E]+', '', text)  # حذف کاراکترهای غیر قابل چاپ و کاراکتر NULL
#     text = re.sub(r'\s+', ' ', text)  # کاهش فاصله‌های خالی متوالی به یک فاصله
#     return text.strip()  # حذف فاصله‌های خالی در ابتدای و انتهای متن

# # مرحله 1: تمیز کردن متن
# file_content = clean_text(file_content)
# file_content = remove_special_characters(file_content)
# len(file_content)

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>بخش ۱: تقسیم‌بندی متن</title>
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b>۱. تقسیم‌بندی متن یا فایل PDF به بخش‌های کوچک</b>
        <pre>
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
chunks = text_splitter.split_text(file_content)
        </pre>
        <p><b>توضیح:</b></p>
        <ul>
            <li><b>هدف:</b> تقسیم متن بزرگ به بخش‌های کوچک‌تر برای پردازش بهتر.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>RecursiveCharacterTextSplitter</code>: کلاس برای تقسیم متن به بخش‌های کوچکتر.</li>
                    <li><code>chunk_size=1500</code>: حداکثر تعداد کاراکترها در هر بخش.</li>
                    <li><code>chunk_overlap=150</code>: تعداد کاراکترهای تکراری بین بخش‌ها.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ایجاد نمونه از <code>RecursiveCharacterTextSplitter</code>.</li>
                    <li>تقسیم متن با استفاده از متد <code>split_text</code>.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [ ]:
# Splitting Text or PDF  into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
chunks = text_splitter.split_text(file_content)


<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>بخش ۲: ایجاد Vector Store</title>
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b>۲. ایجاد Vector Store با استفاده از Chroma</b>
        <pre>
vector_store = Chroma.from_texts(chunks, embed_model)
        </pre>
        <p><b>توضیح:</b></p>
        <ul>
            <li><b>هدف:</b> ذخیره‌سازی برداری از بخش‌های متنی برای جستجو و بازیابی سریع‌تر.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>Chroma</code>: سیستم مدیریت بردار برای ذخیره‌سازی و بازیابی بردارهای متنی.</li>
                    <li><code>from_texts</code>: متد تبدیل متون به بردارهای عددی.</li>
                    <li><code>chunks</code>: لیست بخش‌های متنی تقسیم‌بندی شده.</li>
                    <li><code>embed_model</code>: مدل امبدینگ برای تبدیل متن به بردار.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>تبدیل هر بخش از <code>chunks</code> به بردار با استفاده از <code>embed_model</code>.</li>
                    <li>ذخیره بردارها در <code>Chroma</code> برای جستجو و بازیابی سریع.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [ ]:
# Creating Vector Store using Chroma
vector_store = Chroma.from_texts(chunks, embed_model)

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>بخش ۳: ایجاد Retriever</title>
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b>۳. ایجاد Retriever</b>
        <pre>
retriever = vector_store.as_retriever()
        </pre>
        <p><b>توضیح:</b></p>
        <ul>
            <li><b>هدف:</b> فراهم کردن امکان بازیابی بخش‌های مرتبط از بردارهای ذخیره شده.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>vector_store</code>: پایگاه داده برداری.</li>
                    <li><code>as_retriever()</code>: متد ایجاد بازیاب از <code>vector_store</code>.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ایجاد شیء بازیاب با استفاده از <code>as_retriever()</code>.</li>
                    <li>استفاده از بازیاب برای یافتن بخش‌های مرتبط بر اساس پرسش.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [ ]:
# Creating Retriever
retriever = vector_store.as_retriever()

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>بخش ۴: ایجاد Retrieval Chain</title>
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b>۴. ایجاد Retrieval Chain با استفاده از LLM و Retriever</b>
        <pre>
chain = create_retrieval_chain(combine_docs_chain=llm, retriever=retriever)
        </pre>
        <p><b>توضیح:</b></p>
        <ul>
            <li><b>هدف:</b> ایجاد زنجیره پردازشی که از مدل زبانی بزرگ و بازیاب استفاده کند.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>create_retrieval_chain</code>: تابع ایجاد زنجیره پردازشی.</li>
                    <li><code>combine_docs_chain=llm</code>: استفاده از مدل زبانی برای پردازش اسناد.</li>
                    <li><code>retriever</code>: بازیاب اطلاعات.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ترکیب قابلیت‌های بازیاب و مدل زبانی در یک زنجیره.</li>
                    <li>استفاده از زنجیره برای تولید پاسخ‌های دقیق‌تر به پرسش‌های کاربران.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [ ]:
# Creating Retrieval Chain using LLM and Retriever
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>بخش ۵: Retrieval-QA Chat Prompt</title>
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b>۵. Retrieval-QA Chat Prompt</b>
        <pre>
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
        </pre>
        <p><b>توضیح:</b></p>
        <ul>
            <li><b>هدف:</b> بارگذاری پرامپت چت سوال و جواب از مخزن LangChain.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>hub.pull</code>: متد دانلود منابع از مخزن.</li>
                    <li><code>"langchain-ai/retrieval-qa-chat"</code>: مسیر مخزن پرامپت.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>دانلود پرامپت با استفاده از <code>hub.pull</code>.</li>
                    <li>استفاده از پرامپت در مدل زبانی برای تعاملات سوال و جواب.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [9]:
# Retrieval-QA Chat Prompt
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

/Users/majidjafari/Documents/proposal/code/AhmadRAG_env/lib/python3.11/site-packages/langsmith/client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>بخش ۶: Combining Documents</title>
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 10px;
            border-radius: 5px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 40px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b>۶. Combining Documents</b>
        <pre>
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
        </pre>
        <p><b>توضیح:</b></p>
        <ul>
            <li><b>هدف:</b> ترکیب اسناد بازیابی شده برای ایجاد پاسخ‌های دقیق‌تر.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>create_stuff_documents_chain</code>: تابع ایجاد زنجیره ترکیب اسناد.</li>
                    <li><code>llm</code>: مدل زبانی بزرگ.</li>
                    <li><code>retrieval_qa_chat_prompt</code>: پرامپت برای تعاملات QA Chat.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ایجاد زنجیره ترکیب اسناد با استفاده از <code>llm</code> و پرامپت.</li>
                    <li>پردازش اسناد بازیابی شده برای تولید پاسخ نهایی.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [ ]:
# Combining Documents
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

<!DOCTYPE html>
<html lang="fa" dir="rtl">
<head>
    <div style="direction:rtl; line-height:300%;">
        <font face="B Nazanin" size="5">
    <meta charset="UTF-8">
    <title>بخش ۷: Final Retrieval Chain</title>
    <style>
        body {
            direction: rtl;
            line-height: 300%;
            font-family: 'B Nazanin', sans-serif;
            font-size: 20px;
            padding: 20px;
            background-color: #ffffff;
            color: #333333;
        }
        pre {
            background-color: #FFC0CB;
            padding: 1px;
            border-radius: 1px;
            direction: ltr;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
            font-size: 20px;
            overflow-x: auto;
            margin-bottom: 20px;
        }
        code {
            background-color: #f4f4f4;
            padding: 1px 1px;
            border-radius: 4px;
            font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace;
        }
        strong {
            font-weight: bold;
        }
        ul {
            padding-right: 10px;
            margin-bottom: 10px;
        }
        a {
            color: #2980b9;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div>
        <b>۷. Final Retrieval Chain</b>
        <pre>
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
        </pre>
        <p><b>توضیح:</b></p>
        <ul>
            <li><b>هدف:</b> ایجاد زنجیره نهایی بازیابی برای پاسخ‌های دقیق‌تر.</li>
            <li><b>کامپوننت‌ها:</b>
                <ul>
                    <li><code>create_retrieval_chain</code>: تابع ایجاد زنجیره نهایی.</li>
                    <li><code>retriever</code>: بازیاب اطلاعات.</li>
                    <li><code>combine_docs_chain</code>: زنجیره ترکیب اسناد.</li>
                </ul>
            </li>
            <li><b>فرآیند:</b>
                <ol>
                    <li>ایجاد زنجیره نهایی با ترکیب <code>retriever</code> و <code>combine_docs_chain</code>.</li>
                    <li>بازیابی بخش‌های مرتبط و تولید پاسخ نهایی با استفاده از زنجیره.</li>
                </ol>
            </li>
        </ul>
    </div>
</body>
</html>


In [ ]:
#Final Retrieval Chain
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)    


In [11]:
query1 = "خبرگزاری تسنیم در مورد چه میگوید"
query2 ="سخنگوی دولت کیست؟"
query3 ="معاون اول رئیس جمهور کیست"
query4 = "مریم عبدلعلی کیست"
query5 = " حسین جوهری کیست"

In [12]:
response1_withoutRAG = llm.invoke(query1)
response2_withoutRAG = llm.invoke(query2)
response3_withoutRAG = llm.invoke(query3)
response4_withoutRAG = llm.invoke(query4)
response5_withoutRAG = llm.invoke(query5)

In [13]:
print("_"*50+"response1______withoutRAG"+"_"*50+"\n")
print(response1_withoutRAG+"\n")
print("_"*50+"response2______withoutRAG"+"_"*50+"\n")
print(response2_withoutRAG+"\n")
print("_"*50+"response3______withoutRAG"+"_"*50+"\n")
print(response3_withoutRAG+"\n")
print("_"*50+"response4______withoutRAG"+"_"*50+"\n")
print(response4_withoutRAG+"\n")
print("_"*50+"response5______withoutRAG"+"_"*50+"\n")
print(response5_withoutRAG+"\n")

__________________________________________________response1______withoutRAG__________________________________________________

بسیار خوشحالم که می توانم به شما در مورد خبرگزاری تسنیم بنویسم.

تسنیم (کدبانکی: TASNIM) یک خبرگزاری ایرانی است که در سال ۱۳۹۰ تأسیس شد. این خبرگزاری از مهم‌ترین مراکز نشر اخبار و اطلاعات در ایران به شمار می‌آید.
تسنیم از اول ماه مهٔ ۱۳۹۲، بر اساس دستورالعمل شماره ۵۱ مورخه ۲ اسفند ۱۳۹۱ وزیر فرهنگ و ارشاد اسلامی، رسماً به عنوان خبرگزاری رسمی کشور اعلام شد.

__________________________________________________response2______withoutRAG__________________________________________________

در جمهوری اسلامی، رئیس قوه قضاییه و نیز رئیس جمهور ایران (یا رئیس دولت) در مقام سخنگو قرار دارند. همچنین رییس مجلس شورای اسلامی هم می‌تواند به عنوان سخنگوی مجلس عمل کند.

__________________________________________________response3______withoutRAG__________________________________________________

رئیس جمهور، نخست وزیر و معاون وی را رئیس جمهور می نامند. و بر اساس قانون اساسی ریاست جمهو

In [14]:
response1_withRAG = retrieval_chain.invoke({"input": query1})
response2_withRAG = retrieval_chain.invoke({"input": query2})
response3_withRAG = retrieval_chain.invoke({"input": query3})
response4_withRAG = retrieval_chain.invoke({"input": query4})
response5_withRAG = retrieval_chain.invoke({"input": query5})

In [15]:
print("_"*50+"response1______with RAG"+"_"*50+"\n")
print(response1_withRAG['answer']+"\n")
print("_"*50+"response2______with RAG"+"_"*50+"\n")
print(response2_withRAG['answer']+"\n")
print("_"*50+"response3______with RAG"+"_"*50+"\n")
print(response3_withRAG['answer']+"\n")
print("_"*50+"response4______with RAG"+"_"*50+"\n")
print(response4_withRAG['answer']+"\n")
print("_"*50+"response5______with RAG"+"_"*50+"\n")
print(response5_withRAG['answer']+"\n")


__________________________________________________response1______with RAG__________________________________________________

خبرگزاری تسنیم در این متن، سه موضوع را پوشش می‌دهد:

1. اعطای جایزه "آزمونِ زمان" مِندِلزون سال ۲۰۲۱ به عضو هیات علمی دانشگاه صنعتی خواجه نصیرالدین طوسی (دکتر حسین جوهری)
2. تأکید رئیس انجمن کاردرمانی ایران بر اهمیت و نقش کاردرمان در بازتوانی حرفه‌ای و فیزیکی افراد ناتوان
3. گزارش از مصوبه‌ی دولت درباره رفع رجیستری آیفون

__________________________________________________response2______with RAG__________________________________________________

سید ستار هاشمی سخنگوی دولت است.

__________________________________________________response3______with RAG__________________________________________________

محمد رضا عارف

__________________________________________________response4______with RAG__________________________________________________

مریم عبدلعلی استاد دانشگاه خواجه نصیرالدین طوسی است. او در سال ۲۰۱۹ دکترای خود را در هوش مصنوعی از دانشگاه امیرکبیر دریافت کرد و